In [1]:
from sympy import Tuple, Matrix

from sympde.core import Constant
from sympde.calculus import grad, dot, inner, cross, rot, curl, div
from sympde.calculus import laplace, hessian
from sympde.topology import (dx, dy, dz)
from sympde.topology import FunctionSpace, VectorFunctionSpace, Derham
from sympde.topology import element_of_space
from sympde.topology import ProductSpace
from sympde.topology import Boundary, NormalVector, TangentVector
from sympde.topology import Domain, Line, Square, Cube
from sympde.topology import Trace, trace_0, trace_1
from sympde.topology import Union
from sympde.expr import BilinearForm, LinearForm
from sympde.expr import Norm, TerminalExpr
from sympde.expr import find, EssentialBC


from psydac.fem.basic   import FemField
from psydac.fem.vector  import VectorFemField
from psydac.api.discretization import discretize
from psydac.fem.vector         import ProductFemSpace
from psydac.linalg.block       import ProductSpace, BlockVector, BlockLinearOperator, BlockMatrix
from psydac.linalg.utilities   import array_to_stencil

from scipy.sparse.linalg import cg, gmres
from scipy.sparse.linalg import spsolve
from scipy import linalg

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import animation

import numpy as np
from sympy import lambdify

import sympy as sym

from psydac.linalg.stencil import StencilMatrix, StencilVectorSpace, StencilVector
from psydac.linalg.kron    import KroneckerStencilMatrix as Stencil_kron
from psydac.fem.vector     import ProductFemSpace
from psydac.fem.splines    import SplineSpace
from psydac.fem.tensor     import TensorFemSpace
from psydac.feec.derivatives import Grad

In [72]:
p = 2
Nel = 6
Nbase = Nel + p
el_b = np.linspace(0., 1., Nel + 1)

In [80]:
V0x = SplineSpace(p, grid=el_b, periodic=False, normalize)
V0y = SplineSpace(p, grid=el_b, periodic=False)
V0z = SplineSpace(p, grid=el_b, periodic=False)

V0_h = TensorFemSpace(V0x, V0y, V0z)

V1_x = V0_h.reduce_degree(axes=[0], normalize=True)
V1_y = V0_h.reduce_degree(axes=[1], normalize=True)
V1_z = V0_h.reduce_degree(axes=[2], normalize=True)

V1 = [V1_x, V1_y, V1_z]

In [81]:
print(V1_x)


> ldim   :: 3
> total nbasis  :: 448
> nbasis :: (7, 8, 8)



In [77]:
test = V0x.reduce_degree(axes=[0], normalize=True)

AttributeError: 'SplineSpace' object has no attribute 'reduce_degree'

In [26]:
V1_x.vector_space.pads

(1, 2, 2)

In [2]:
# ... create model with method of manufactured solutions
x, y, z = sym.symbols('x, y, z')

u = x*sym.sin(2*sym.pi*x)*y*(1 - sym.cos(2*sym.pi*y))*z*sym.sin(2*sym.pi*z)

f = -sym.diff(u, x, x) -sym.diff(u, y, y) - sym.diff(u, z, z)
#f = sym.lambdify((x, y, z), f)
u = sym.lambdify((x, y, z), u)
# ...

In [3]:
f

-4*pi*x*y*(pi*z*sin(2*pi*z) - cos(2*pi*z))*(cos(2*pi*y) - 1)*sin(2*pi*x) - 4*pi*x*z*(pi*y*cos(2*pi*y) + sin(2*pi*y))*sin(2*pi*x)*sin(2*pi*z) - 4*pi*y*z*(pi*x*sin(2*pi*x) - cos(2*pi*x))*(cos(2*pi*y) - 1)*sin(2*pi*z)

In [23]:
ncells = [5, 5, 5]
degree = [2, 2, 2]

# ... abstract model
domain = Cube()

derham = Derham(domain)
V0, V1, V2, V3 = derham.spaces

F = element_of_space(V2, name='F')

p, q = [element_of_space(V1, name=i) for i in ['p', 'q']]
u, v = [element_of_space(V0, name=i) for i in ['u', 'v']]

a  = BilinearForm(((p,u),(q,v)), dot(p,q) + dot(grad(u),q) - dot(p,grad(v)))
l = LinearForm((q,v), f*v)

domain_h = discretize(domain, ncells=ncells)
derham_h = discretize(derham, domain_h, degree=degree)

V0_h, V1_h, V2_h, V3_h = derham_h.spaces

Xh = V1_h * V0_h

ah = discretize(a, domain_h, [Xh, Xh])
lh = discretize(l, domain_h, Xh)


GRAD, CURL, DIV = derham_h.derivatives_as_matrices

In [49]:
import psydac

test = psydac.feec.derivatives.d_matrix(5, 2, False)

In [53]:
test.toarray()

array([[-1.,  1.,  0.,  0.,  0.],
       [ 0., -1.,  1.,  0.,  0.],
       [ 0.,  0., -1.,  1.,  0.],
       [ 0.,  0.,  0., -1.,  1.]])

In [27]:
a  = BilinearForm(((p),(q)), dot(p,q))



ah = discretize(a, domain_h, [V1_h, V1_h])

In [28]:
M11 = ah.assemble()

In [24]:
M1 = ah.assemble()

In [37]:
M1.blocks[0][0].dot(GRAD.blocks[0])

AssertionError: 

In [40]:
M1.blocks[0][0].dot?

In [ ]:
M1.blocks[0][0].dot

In [36]:
M1[0][0]

AssertionError: 

In [28]:
M.blocks

((<psydac.linalg.stencil.StencilMatrix at 0x7fe7378612e8>,
  None,
  None,
  <psydac.linalg.stencil.StencilMatrix at 0x7fe737861128>),
 (None,
  None,
  <psydac.linalg.stencil.StencilMatrix at 0x7fe7378613c8>),
 (None,
  None,
  <psydac.linalg.stencil.StencilMatrix at 0x7fe737861470>),
 (<psydac.linalg.stencil.StencilMatrix at 0x7fe7378614a8>,
  None))

In [40]:
M[0]

AssertionError: 

In [43]:
TerminalExpr(a)

[DomainExpression(Cube, Matrix([
 [p[0]*q[0],         0,         0],
 [        0, p[1]*q[1],         0],
 [        0,         0, p[2]*q[2]]]))]